# Analyze results
Aug 28, 2020

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/')
from modules_image_analysis import *


In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [7]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [8]:
dict1={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [41]:
# parent_dir=u.result
parent_dir=dict1['128']
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20201202_094646_cgan_model2', '20201002_073628', '20…

In [42]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201205_050122_cgan_model3


In [11]:
# ### Read loss data
# with open (result_dir+'/metrics.pickle','rb') as f:
#     metrics_dict=pickle.load(f)
    
# df_metrics=pd.DataFrame([])
# for key in metrics_dict.keys(): df_metrics[key]=metrics_dict[key]
# df_metrics['step']=np.arange(df_metrics.shape[0])
# # df_metrics=df_metrics.dropna()

## Plot Losses

In [43]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [44]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
82791,82791.0,19.0,0.165133,0.077813,0.242947,6.234944,310.091400,303.856445,-4.007738,320.228210,-4.026021,4.877138,-6.610165,-6.175985,0.163800
82792,82792.0,19.0,0.160400,0.313488,0.473888,6.119004,331.038086,324.919067,-5.047490,318.948181,-5.335797,6.318330,-6.671207,-6.065934,0.154950
82793,82793.0,19.0,0.219914,0.191389,0.411303,7.769037,315.309326,307.540283,-5.083526,317.365112,-5.264512,6.229833,-4.616519,-7.765447,0.155353
82794,82794.0,19.0,0.135867,0.097953,0.233820,7.912117,318.287750,310.375641,-5.003528,313.743652,-5.093824,4.627384,-7.790408,-7.896792,0.156363
82795,82795.0,19.0,0.231231,0.110014,0.341245,5.700191,314.336090,308.635895,-5.211887,313.077393,-4.885977,5.045421,-7.150939,-5.641085,0.160391
82796,82796.0,19.0,0.096720,0.264730,0.361450,7.706915,315.281464,307.574554,-4.156780,316.117218,-4.294625,6.364378,-4.948901,-7.694006,0.153739
82797,82797.0,19.0,0.119453,0.111055,0.230508,7.680281,319.305145,311.624878,-4.412892,316.648102,-4.301264,4.222116,-8.181784,-7.672161,0.156203
82798,82798.0,19.0,0.089403,0.123988,0.213391,6.953867,319.940979,312.987122,-3.200492,314.967926,-3.182302,5.268039,-6.522677,-6.929627,0.154892
82799,82799.0,19.0,0.177844,0.165654,0.343499,6.636680,313.853455,307.216766,-2.379458,316.619019,-2.448957,5.543026,-5.825879,-6.604492,0.157968
82800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,316.137817,-2.414024,NaN,NaN,NaN,NaN


In [14]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [15]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

-9.382873802185058


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
56104,56104.0,13.0,0.076500,0.093268,0.169768,4.064933,321.131378,317.066437,-9.149649,309.418945,-10.487954,4.229947,-3.909312,-4.045177,0.140677
53891,53891.0,13.0,0.110777,0.078651,0.189428,4.225865,314.827545,310.601685,-6.029517,309.938721,-10.281973,4.178424,-4.312673,-4.208934,0.143790
49222,49222.0,11.0,0.087574,0.083537,0.171112,4.111817,315.697510,311.585693,-6.279222,312.483551,-10.227720,4.483982,-3.890503,-4.093879,0.144548
56078,56078.0,13.0,0.092146,0.088691,0.180837,4.500654,313.757782,309.257141,-7.591362,307.130096,-10.189348,4.375178,-4.486544,-4.489113,0.139385
49496,49496.0,11.0,0.079212,0.085679,0.164891,4.335611,316.718506,312.382904,-8.010885,308.880768,-10.108874,5.087113,-4.220433,-4.320971,0.142592
54321,54321.0,13.0,0.074255,0.078513,0.152768,4.421567,313.180542,308.758972,-8.553538,310.522430,-10.046755,4.376964,-4.536149,-4.408215,0.141977
68235,68235.0,16.0,0.087294,0.080731,0.168024,4.255686,314.535461,310.279785,-8.324650,308.082672,-10.004930,4.386054,-4.139435,-4.239946,0.138278
68145,68145.0,16.0,0.105074,0.077201,0.182275,4.133782,308.751221,304.617432,-8.600389,309.809906,-9.988571,3.953754,-4.289287,-4.116913,0.140080
54560,54560.0,13.0,0.096191,0.082560,0.178751,4.282816,317.082062,312.799255,-8.089333,319.473053,-9.976769,4.491848,-4.189195,-4.268003,0.150377
49109,49109.0,11.0,0.071555,0.071742,0.143297,4.532667,315.977203,311.444550,-7.284949,310.581451,-9.957847,4.389954,-4.583495,-4.519838,0.145291


In [16]:
display(df_metrics.sort_values(by=['hist_chi']).head(8))
display(df_metrics.sort_values(by=['spec_chi']).head(8))

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
56104,56104.0,13.0,0.076500,0.093268,0.169768,4.064933,321.131378,317.066437,-9.149649,309.418945,-10.487954,4.229947,-3.909312,-4.045177,0.140677
53891,53891.0,13.0,0.110777,0.078651,0.189428,4.225865,314.827545,310.601685,-6.029517,309.938721,-10.281973,4.178424,-4.312673,-4.208934,0.143790
49222,49222.0,11.0,0.087574,0.083537,0.171112,4.111817,315.697510,311.585693,-6.279222,312.483551,-10.227720,4.483982,-3.890503,-4.093879,0.144548
56078,56078.0,13.0,0.092146,0.088691,0.180837,4.500654,313.757782,309.257141,-7.591362,307.130096,-10.189348,4.375178,-4.486544,-4.489113,0.139385
49496,49496.0,11.0,0.079212,0.085679,0.164891,4.335611,316.718506,312.382904,-8.010885,308.880768,-10.108874,5.087113,-4.220433,-4.320971,0.142592
54321,54321.0,13.0,0.074255,0.078513,0.152768,4.421567,313.180542,308.758972,-8.553538,310.522430,-10.046755,4.376964,-4.536149,-4.408215,0.141977
68235,68235.0,16.0,0.087294,0.080731,0.168024,4.255686,314.535461,310.279785,-8.324650,308.082672,-10.004930,4.386054,-4.139435,-4.239946,0.138278
68145,68145.0,16.0,0.105074,0.077201,0.182275,4.133782,308.751221,304.617432,-8.600389,309.809906,-9.988571,3.953754,-4.289287,-4.116913,0.140080


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
56324,56324.0,13.0,0.089864,0.088192,0.178057,4.384623,320.175079,315.790466,-6.788898,292.940125,-7.883965,4.251951,-4.372250,-4.371335,0.146141
70767,70767.0,17.0,0.091550,0.095155,0.186705,4.862798,312.733887,307.871094,-1.400028,292.965363,-1.025736,5.201448,-5.029331,-4.854022,0.154866
56327,56327.0,13.0,0.080090,0.082645,0.162735,4.553821,310.596252,306.042419,-6.654824,293.260773,-8.605462,4.499014,-4.535455,-4.542938,0.139842
64065,64065.0,15.0,0.080864,0.083313,0.164177,4.017734,313.739807,309.722076,-1.917297,294.363892,-1.631511,3.740299,-4.193769,-3.998411,0.140985
74928,74928.0,18.0,0.089786,0.086676,0.176462,4.183827,309.484344,305.300507,0.588306,294.403687,0.796650,3.929134,-4.357525,-4.167441,0.141376
56416,56416.0,13.0,0.070516,0.088591,0.159107,4.044260,313.473877,309.429626,-7.178848,294.532288,-8.180719,4.359211,-4.066434,-4.025572,0.137641
56325,56325.0,13.0,0.090565,0.082505,0.173070,4.407179,316.285400,311.878235,-6.820634,294.761536,-6.655887,4.699009,-4.301916,-4.394319,0.137611
78267,78267.0,18.0,0.074619,0.079912,0.154530,4.191510,306.764069,302.572571,2.566254,295.079742,3.320934,4.273508,-4.190269,-4.175963,0.138684


In [17]:
df_metrics[df_metrics.step.isin([44861,44862,44863])]

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
44861,44861.0,10.0,0.101772,0.079216,0.180988,4.162025,315.317688,311.155670,-9.360899,313.040283,-8.899093,4.420756,-4.036863,-4.144569,0.141475
44862,44862.0,10.0,0.093893,0.166928,0.260821,5.397851,320.895630,315.497772,-9.455900,315.021881,-8.857945,4.044906,-3.707994,-5.390597,0.141896
44863,44863.0,10.0,0.186349,0.088439,0.274787,3.482584,311.257416,307.774841,-9.663067,326.675873,-8.280173,2.236618,-5.415519,-3.445221,0.150763


## Read stored chi-squares for images

In [45]:

df_list=[]
label_list=[0,1,2,3]
for label in label_list:
    df=pd.read_pickle(result_dir+'/df_processed_{0}.pkle'.format(str(label)))
    df[['epoch','step']]=df[['epoch','step']].astype(int)
    df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting
    df_list.append(df)

col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']



In [46]:
# df_list[-1]

In [47]:
def f_slice_df(df_list,label,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    df=df_list[label]
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [48]:
w=interactive(f_slice_df,df_list=fixed(df_list),label=label_list,
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(Dropdown(description='label', options=(0, 1, 2, 3), value=0), FloatSlider(value=0.2, des…

In [26]:
df_sliced=w.result
df_sliced

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
416,5-20850,0.159772,4.430273,30.868725,0.030906,0.076781,0.052085,0.007246,13.693949,456.161590,5,20850
369,4-18500,0.173009,4.412035,30.946736,0.045926,0.052714,0.074369,0.039150,15.265462,386.997154,4,18500
247,2-12400,0.179059,2.246898,28.966460,0.049838,0.069997,0.059223,0.033405,12.974771,164.150530,2,12400
297,3-14900,0.179504,4.220739,28.674201,0.038987,0.082226,0.058291,0.018696,13.389783,523.536700,3,14900
210,2-10550,0.188447,4.514675,27.498857,0.036028,0.086159,0.066260,0.030041,13.191298,372.945943,2,10550
352,4-17650,0.188977,3.493567,30.210712,0.049723,0.072804,0.066451,0.011019,13.593824,394.642895,4,17650
540,6-27050,0.194045,1.856233,32.168782,0.043730,0.068180,0.082134,0.023306,12.846263,181.807363,6,27050
537,6-26900,0.200988,2.450517,28.553127,0.014800,0.092998,0.093190,0.008015,12.391132,242.244269,6,26900
238,2-11950,0.202537,2.344633,29.789321,0.052222,0.066940,0.083375,0.039899,13.188530,191.783302,2,11950
371,4-18600,0.205919,3.660255,29.165244,0.039432,0.074085,0.092401,0.022756,13.475450,424.551821,4,18600


In [56]:
best_step=[]
label=1
best_step.append(f_slice_df(df_list,label,cutoff=0.4,sort_col='chi_1',head=2,display_flag=False,epoch_range=[7,None]).step.values)
best_step.append(f_slice_df(df_list,label,cutoff=0.8,sort_col='chi_1c',head=2,display_flag=False,epoch_range=[7,None]).step.values)
best_step.append(f_slice_df(df_list,label,cutoff=0.4,sort_col='chi_spec1',head=2,display_flag=False,epoch_range=[7,None]).step.values)

best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[7, None]
{'chi_1': 0.34020083776100696, 'chi_spec1': 3.170250267602132}
[7, None]
{'chi_1': 0.4669880657972603, 'chi_spec1': 6.734376935183829}
[7, None]
{'chi_1': 0.34020083776100696, 'chi_spec1': 3.170250267602132}
[32950 33000 35250 40450 43050 64250]


In [57]:
# best_step=[40800]
# best_step=np.arange(40130,40135).astype(int)

In [58]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(6, 22)
[(7, 32950), (7, 33000), (8, 35250), (9, 40450), (10, 43050), (15, 64250)]


In [51]:
df_best[col_list]

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
372,4-18650,0.173242,0.738616,33.314278,0.051909,0.023971,0.097362,0.043625,8.247023,87.809745,4,18650
383,4-19200,0.167513,1.893344,29.441005,0.070196,0.067439,0.029878,0.040894,6.528057,156.748093,4,19200
455,5-22800,0.438699,2.257535,31.424140,0.127356,0.020699,0.290645,0.102057,8.174378,371.578471,5,22800
456,5-22850,0.169310,2.295692,31.153716,0.021990,0.019205,0.128115,0.003602,7.653064,442.453291,5,22850
478,5-23950,0.821160,1.194288,30.407599,0.096068,0.013855,0.711237,0.077093,9.135314,201.825040,5,23950
658,7-32950,3.128956,3.225733,34.957144,0.268920,0.193527,2.666509,0.024363,11.564921,253.857921,7,32950


### Plots 2: stored data

In [52]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)
# ### Extract validation data
# fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_{0}_train_val.npy'.format(sigma_list[label])
# s_val=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:]

# print(s_val.shape)

# ### Compute histogram and spectrum of raw data 
# dict_val=post.f_compute_hist_spect(s_val,bins)
# del(s_val)
sigma_list=[0.5,0.65,0.8,1.1]
label_list=[0,1,2,3]
bkgnd_dict={}
num_bkgnd=2000

for label in label_list:
    fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_{0}_train_val.npy'.format(sigma_list[label])
    samples=np.load(fname,mmap_mode='r+')[-num_bkgnd:][:,0,:,:]
    
    dict_val=post.f_compute_hist_spect(samples,bins)
#     print(dict_val)
    bkgnd_dict[str(label)]=dict_val
    del samples

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [53]:

def f_plot_hist_spec(df_list,class_labels,steps_list,bkg_dict,plot_type):

    assert plot_type in ['hist','spec','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec']:     fig=plt.figure(figsize=(6,6))
    for class_label in class_labels:
        df=df_list[class_label]
        df=df[df.step.isin(steps_list)]
        print(df.shape)
        for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
            label=row.label
            if plot_type=='hist':
                x1=row.hist_bin_centers
                y1=row.hist_val
                yerr1=row.hist_err
                x1=f_invtransform(x1)
                
                plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            if plot_type=='spec':
                y2=row.spec_val
                yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
                x2=np.arange(len(y2))

                plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
                plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

            if plot_type=='grid':
                images=np.load(row.fname)[:,:,:]
                f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
        ### Plot reference data
        dict_bkg=bkg_dict[str(class_label)]
        if plot_type=='hist':
            x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
            x=f_invtransform(x)
            plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
            plt.title('Pixel Intensity Histogram')
            plt.xscale('symlog',linthreshx=50)
        
        if plot_type=='spec':
            y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
            x=np.arange(len(y))
            plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
            plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    

# f_plot_hist_spec(df_list,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [54]:
label_list

[0, 1, 2, 3]

In [59]:
def f_widget_compare(df_list,class_labels,steps_list,bkgnd_dict,plot_type):
    f_plot_hist_spec(df_list,class_labels,steps_list,bkgnd_dict,plot_type)
    
interact_manual(f_widget_compare,df_list=fixed(df_list),
                class_labels=SelectMultiple(options=label_list),
                steps_list=SelectMultiple(options=best_step),
                bkgnd_dict=fixed(bkgnd_dict),plot_type=ToggleButtons(options=['hist','spec','grid']))

interactive(children=(SelectMultiple(description='class_labels', options=(0, 1, 2, 3), value=()), SelectMultip…

<function __main__.f_widget_compare(df_list, class_labels, steps_list, bkgnd_dict, plot_type)>